In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split , StratifiedKFold,KFold
from sklearn import model_selection
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.cluster import DBSCAN,KMeans
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.metrics.cluster import homogeneity_score
from sklearn.metrics.cluster import completeness_score
from matplotlib.colors import ListedColormap
from sklearn.preprocessing import LabelEncoder
import scipy.spatial
import warnings
warnings.filterwarnings('ignore')

# Question 1 (15 pts)

Implement the ADABoost algorithm by using the scikit implementation of the logistic regression. Evaluate the result on a real dataset between a single logistic regression and AdaBoost (use K-Fold cross validation).

This links can be helpful: http://rob.schapire.net/papers/explaining-adaboost.pdf and https://en.wikipedia.org/wiki/AdaBoost


# Loading dataset

In [2]:
breast_cancer = datasets.load_breast_cancer()
X = pd.DataFrame(breast_cancer.data, columns=breast_cancer.feature_names)
y = pd.Categorical.from_codes(breast_cancer.target, breast_cancer.target_names)
labelencoder_Y = LabelEncoder()
Y = labelencoder_Y.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(np.array(X), np.array(Y), test_size=0.3,random_state=10)

# AdaBoost from scratch

## Definition of AdaBoost function which can compare the result of logistic regreesion and AdaBoost in each fold and finally reports the cross validation accuracy for both logistic regression and AdaBoost models

In [3]:
def AdaBoost_scratch(X_train,y_train, M=10, learning_rate = 1):
    #Initialization of utility variables
    
    skf = KFold(n_splits=5,random_state=10,shuffle=True )
    acc_train = []
    acc_valid =[]
    acc_valid_lg = [] 
    k=1
    #estimator_weight_list_tot=[]
    for train_index, valid_index in skf.split(X_train, y_train):
        _X_train_ = X_train[train_index]
        _X_valid_ = X_train[valid_index]
        _y_train_ = y_train[train_index]
        _y_valid_ = y_train[valid_index]
        
        lg_valid=LogisticRegression()
        clf = LogisticRegression(random_state=10).fit(_X_train_, _y_train_)
        print('Accuracy for logistic regression for fold',k,'is:',clf.score(_X_valid_, _y_valid_))
        acc_valid_lg.append(clf.score(_X_valid_, _y_valid_))
        
        N1 = len(_y_valid_)
        estimator_list, y_predict_list_train,y_predict_list_valid, estimator_error_list, estimator_weight_list, sample_weight_list = [], [],[],[],[],[]
        N2= len(_y_train_)
        #Initialize the sample weights
        sample_weight = np.ones(N2) / N2
        sample_weight_list.append(sample_weight.copy())
        #estimator_weight_list=[]
        
        #For m = 1 to M
        for m in range(M):
            #Fit a classifier
            estimator = LogisticRegression()
            estimator.fit(_X_train_, _y_train_, sample_weight=sample_weight)
            y_predict_train = estimator.predict(_X_train_)
            y_predict_valid = estimator.predict(_X_valid_)

            #Misclassifications
            
            incorrect = (y_predict_train != _y_train_)

            #Estimator error
            estimator_error = np.mean( np.average(incorrect, weights=sample_weight, axis=0))

            #Boost estimator weights
            estimator_weight =  learning_rate * np.log((1. - estimator_error) / estimator_error)

            #Boost sample weights
            sample_weight *= np.exp(estimator_weight * incorrect * ((sample_weight > 0) | (estimator_weight < 0)))

            #Save iteration values
            estimator_list.append(estimator)
            y_predict_list_train.append(y_predict_train.copy())
            y_predict_list_valid.append(y_predict_valid.copy())
            estimator_error_list.append(estimator_error.copy())
            estimator_weight_list.append(estimator_weight.copy())
            sample_weight_list.append(sample_weight.copy())
        
        
        


        #Convert to np array for convenience   
        estimator_list = np.asarray(estimator_list)
        y_predict_list_train = np.asarray(y_predict_list_train)
        y_predict_list_valid = np.asarray(y_predict_list_valid)
        estimator_error_list = np.asarray(estimator_error_list)
        estimator_weight_list = np.asarray(estimator_weight_list)
        sample_weight_list = np.asarray(sample_weight_list)
        

        #Predictions
        preds_valid = (np.array([np.sign((y_predict_list_valid[:,point] * estimator_weight_list).sum()) for point in range(N1)]))
        
        print('Accuracy for AdaBoost for fold',k,'is:',(preds_valid == _y_valid_).sum() / N1)
        k=k+1
        acc_valid.append((preds_valid == _y_valid_).sum() / N1)
        preds_train = (np.array([np.sign((y_predict_list_train[:,point] * estimator_weight_list).sum()) for point in range(N2)]))
        acc_train.append((preds_train == _y_train_).sum() / N2)
        print('--------------------------------------------------------------------------')
    #y_predict    
    #preds = (np.array([np.sign((y_predict_list[:,point] * estimator_weight_list).sum()) for point in range(N)]))
    

        
        #print('Accuracy = ', (preds == y).sum() / N)
    print('------------------------------------------------------------------------------------------------')
    print('$Accuracy for AdaBoost model$')
    print("Accuracy of Training data AdaBoost:",np.mean(acc_train))
    print("Accuracy of K-fold cross validation AdaBoost:",np.mean(acc_valid))
    print('------------------------------------------------------------------------------')
    print('$Accuracy for Logistic Regression model$')
    print("Accuracy of K-fold cross validation Logistic regression:",np.mean(acc_valid_lg))
    return estimator_list, estimator_weight_list, sample_weight_list

In [4]:
M=4
estimator_list, estimator_weight_list, sample_weight_list=AdaBoost_scratch(X_train,y_train, M, learning_rate = 0.05)

Accuracy for logistic regression for fold 1 is: 0.8875
Accuracy for AdaBoost for fold 1 is: 0.875
--------------------------------------------------------------------------
Accuracy for logistic regression for fold 2 is: 0.9875
Accuracy for AdaBoost for fold 2 is: 0.95
--------------------------------------------------------------------------
Accuracy for logistic regression for fold 3 is: 0.9375
Accuracy for AdaBoost for fold 3 is: 0.925
--------------------------------------------------------------------------
Accuracy for logistic regression for fold 4 is: 0.9620253164556962
Accuracy for AdaBoost for fold 4 is: 0.9873417721518988
--------------------------------------------------------------------------
Accuracy for logistic regression for fold 5 is: 0.9367088607594937
Accuracy for AdaBoost for fold 5 is: 0.9240506329113924
--------------------------------------------------------------------------
--------------------------------------------------------------------------------------

# Accuracy of AdaBoost model for testing data

In [5]:
y_predict=[]
for i in range(0,M):
    y_predict.append(estimator_list[i].predict(X_test))
y_predict_list_test = np.asarray(y_predict)
preds_test = (np.array([np.sign((y_predict_list_test[:,point] * estimator_weight_list).sum()) for point in range(len(y_test))]))
acc_test=(preds_test == y_test).sum() / len(y_test)
print("Accuracy of Testing data:",acc_test)

Accuracy of Testing data: 0.9415204678362573


# We can run AdaBoost Sklearn to compare the result with our code

In [6]:
kfold = model_selection.KFold(n_splits=5, random_state=10)
lg=LogisticRegression()
clf=AdaBoostClassifier(base_estimator=lg,n_estimators=10,learning_rate=1)
results = model_selection.cross_val_score(clf, X_train, y_train, cv=kfold)

#clf.fit(X, y)
print(results.mean())

0.934620253164557


In [7]:
model = clf.fit(X_train, y_train)
y_pred_train=model.predict(X_train)
y_pred = model.predict(X_test)

In [8]:
y_predtion = model_selection.cross_val_predict(clf,X_test,y_test,cv=kfold)
metrics.accuracy_score(y_test, y_predtion)

0.9590643274853801

In [9]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9415204678362573


## *We can see the accuracy for testing data for Sklearn AdaBoost model is same as our AdaBoost model*

# Now we can run logistic regression model to compare it with AdaBoost model

#  Accuracy of Logistic model for testing data

In [10]:
kfold = model_selection.KFold(n_splits=5, random_state=10)
lg=LogisticRegression()
results_lg = model_selection.cross_val_score(lg, X_train, y_train, cv=kfold)
print(results_lg.mean())

0.9321202531645569


In [11]:
model_lg = lg.fit(X_train, y_train)
y_pred_train=model_lg.predict(X_train)
y_pred = model_lg.predict(X_test)
print("Accuracy train:",metrics.accuracy_score(y_train, y_pred_train))
print("Accuracy test:",metrics.accuracy_score(y_test, y_pred))

Accuracy train: 0.9547738693467337
Accuracy test: 0.9473684210526315


# As we can see the accuracy of logistic regression model for our testing data is almost same as the AdaBoost model

# We can run the whole procedure for another dataset

In [12]:
X, y = make_classification(n_samples=1000, n_features=2,n_informative=2, n_redundant=0,random_state=0, shuffle=False)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=20)

In [13]:
M=25
estimator_list, estimator_weight_list, sample_weight_list=AdaBoost_scratch(X_train,y_train, M, learning_rate = 0.001)

Accuracy for logistic regression for fold 1 is: 0.9625
Accuracy for AdaBoost for fold 1 is: 0.9625
--------------------------------------------------------------------------
Accuracy for logistic regression for fold 2 is: 0.9375
Accuracy for AdaBoost for fold 2 is: 0.93125
--------------------------------------------------------------------------
Accuracy for logistic regression for fold 3 is: 0.9625
Accuracy for AdaBoost for fold 3 is: 0.94375
--------------------------------------------------------------------------
Accuracy for logistic regression for fold 4 is: 0.9375
Accuracy for AdaBoost for fold 4 is: 0.95
--------------------------------------------------------------------------
Accuracy for logistic regression for fold 5 is: 0.95
Accuracy for AdaBoost for fold 5 is: 0.95
--------------------------------------------------------------------------
------------------------------------------------------------------------------------------------
$Accuracy for AdaBoost model$
Accurac

In [14]:
y_predict=[]
for i in range(0,M):
    y_predict.append(estimator_list[i].predict(X_test))
y_predict_list_test = np.asarray(y_predict)
preds_test = (np.array([np.sign((y_predict_list_test[:,point] * estimator_weight_list).sum()) for point in range(len(y_test))]))
acc_test=(preds_test == y_test).sum() / len(y_test)
print("Accuracy of Testing data:",acc_test)

Accuracy of Testing data: 0.935


In [15]:
lg=LogisticRegression()
model_lg = lg.fit(X_train, y_train)
y_pred_train=model_lg.predict(X_train)
y_pred = model_lg.predict(X_test)
print("Accuracy train:",metrics.accuracy_score(y_train, y_pred_train))
print("Accuracy test:",metrics.accuracy_score(y_test, y_pred))

Accuracy train: 0.9525
Accuracy test: 0.945


# Again, we can see that both models can predict with high accuracy